In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
# Step 1: Install and import libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# Check TensorFlow version
print("TensorFlow Version:", tf.__version__)


In [ ]:
# Step 2: Load the Oxford Pets dataset (images + labels only)
(ds_train, ds_val), ds_info = tfds.load(
    'oxford_iiit_pet',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,  # returns (image, label) pairs
    with_info=True
)

print("Dataset loaded successfully!")
print("Number of training samples:", tf.data.experimental.cardinality(ds_train).numpy())
print("Number of validation samples:", tf.data.experimental.cardinality(ds_val).numpy())


In [ ]:
import tensorflow as tf

# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0  # normalize to [0,1]
    return image, label

# Apply preprocessing
ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.map(preprocess, num_parallel_calls=AUTOTUNE)

# Shuffle, batch, and prefetch
ds_train = ds_train.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(AUTOTUNE)


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2


In [ ]:
# Load the Oxford-IIIT Pets dataset
(ds_train, ds_val), ds_info = tfds.load(
    'oxford_iiit_pet',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,  # returns (image, label) pairs
    with_info=True
)


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))   # Resize to 224x224
    image = tf.cast(image, tf.float32) / 255.0              # Normalize to [0, 1]
    return image, label

# Train data pipeline
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Validation data pipeline
ds_val = ds_val.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Load pretrained MobileNetV2 base (exclude top)
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Add custom classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(ds_info.features['label'].num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train the model
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10
)


Plot Training Accuracy & Loss

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


Evaluate on Validation Set

In [ ]:
loss, accuracy = model.evaluate(ds_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(f"Validation Loss: {loss:.4f}")


In [ ]:
# Saving Model
# Save in native Keras format
model.save('oxford_pet_model.keras')



Make Predictions on New Images

In [ ]:
!ls


In [ ]:
from PIL import Image
import numpy as np

def predict_image(image_path):
    img = Image.open(image_path).resize((224, 224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    class_idx = np.argmax(prediction)
    class_name = ds_info.features['label'].int2str(class_idx)
    return class_name

# Example: Pass your image path here
print(predict_image('/content/photo(cd).jpg'))



In [ ]:
dog_breeds = [
    'affenpinscher', 'afghan_hound', 'airedale', 'akita', 'appenzeller',
    'australian_terrier', 'basenji', 'beagle', 'blenheim_spaniel',
    'bloodhound', 'border_terrier', 'borzoi', 'boston_bull',
    'boxer', 'brabancon_griffon', 'briard', 'chihuahua',
    'chow', 'clumber', 'cocker_spaniel', 'collie', 'curly_coated_retriever',
    'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'elkhound', 'entlebucher',
    'eskimo_dog', 'flat_coated_retriever', 'french_bulldog', 'german_shepherd',
    'golden_retriever', 'great_pyrenees', 'great_dane', 'greater_swiss_mountain_dog',
    'groenendael', 'ibizan_hound'
]  # short list, Oxford-IIIT has 25 dogs, 12 cats

def predict_image(image_path):
    img = Image.open(image_path).resize((224, 224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    class_idx = np.argmax(prediction)
    class_name = ds_info.features['label'].int2str(class_idx)

    # Check if breed is dog or cat
    category = "Dog" if class_name in dog_breeds else "Cat"
    return f"{category} - Breed: {class_name}"

# Example:
print(predict_image('/content/photo(cd).jpg'))


In [ ]:
print(predict_image('/content/photo(cd2).jpg'))


In [ ]:
import matplotlib.pyplot as plt

def show_prediction(image_path):
    prediction = predict_image(image_path)
    img = Image.open(image_path)

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {prediction}", fontsize=14)
    plt.show()

# Example usage
show_prediction('/content/photo(cd2).jpg')


In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace oxford_pet_classifier.ipynb


In [ ]:
!pip install nbconvert --quiet
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True \
--ClearOutputPreprocessor.enabled=True \
--to notebook --output=cleaned_notebook.ipynb \
oxford_pet_classifier.ipynb


In [ ]:
from google.colab import files
files.download('cleaned_notebook.ipynb')


In [ ]:
!ls
